## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization
def build_mlp(input_shape, output_units=10, num_neurons=[256, 256, 256]):
    """Code Here
    建立你的神經網路
    """
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model


In [6]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [7]:
# 載入 Callbacks
"""Code Here
設定 callbacks: model checkpoint
"""
from keras.callbacks import ModelCheckpoint
model_ckpt = ModelCheckpoint(filepath='./tmp.h5',
                            monitor='val_loss',
                            save_weights_only=True,
                            save_best_only=True)
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
model = build_mlp(x_train.shape[1:])
opt = keras.optimizers.SGD(lr=LEARNING_RATE, momentum=MOMENTUM)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          validation_data=[x_test, y_test],
          callbacks=[model_ckpt],
          shuffle=True)
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]
train_acc = model.history.history["accuracy"]
valid_acc = model.history.history["val_accuracy"]

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
48/49 [============================>.] - ETA: 0s - loss: 2.2759 - accuracy: 0.2556WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x00000139DC40DAF8> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) 

Epoch 43/50
49/49 [==============================] - 4s 87ms/step - loss: 0.8163 - accuracy: 0.7311 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 44/50
49/49 [==============================] - 4s 88ms/step - loss: 0.8063 - accuracy: 0.7346 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 45/50
49/49 [==============================] - 4s 78ms/step - loss: 0.7926 - accuracy: 0.7417 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 46/50
49/49 [==============================] - 4s 79ms/step - loss: 0.7772 - accuracy: 0.7455 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 47/50
49/49 [==============================] - 4s 77ms/step - loss: 0.7663 - accuracy: 0.7503 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 48/50
49/49 [==============================] - 4s 88ms/step - loss: 0.7539 - accuracy: 0.7560 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 49/50
49/49 [==============================] - 3s 69ms/step - loss: 0.7427 - accuracy: 0.7584 

In [13]:
model = keras.model.load_weights('./tmp.h5')
loss_loadback, acc_loadback = model.evaluate(x_test, y_test)

AttributeError: module 'keras' has no attribute 'model'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.hlines(y=loss_loadback, xmin=0, xmax=len(train_loss), colors='r', linestyles='--')
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.hlines(y=acc_loadback, xmin=0, xmax=len(train_loss), colors='r', linestyles='--')
plt.legend()
plt.title("Accuracy")
plt.show()